<a href="https://colab.research.google.com/github/vieira-roberto/dev/blob/main/Aula1_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='Black'> **Aula 1 - Algoritmo Naive Bayes: Criando uma máquina preditiva para análise de risco de crédito** </font>


**Naive Bayes** é o método de classificação mais fácil e rápido disponível e é adequado para lidar com enormes quantidades de informações.

Em várias aplicações, **como filtragem de spam, classificação de texto, análise de sentimento e sistemas de recomendação**, o classificador Naive Bayes tem se mostrado eficaz.


###Dataset de entrada

In [ ]:
## Pacotes
import pandas as pd     #Biblioteca de Funções de Manipulação de Dados

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Importação dos Dados
df = pd.read_csv('/content/cliente_risco_credito.csv', sep=';')

In [ ]:
df

,codigo_cliente,historia_credito,divida,garantia,renda_anual,risco
0,C01,Ruim,Alta,Nenhuma,"< 15.000,00",Alto
1,C02,Desconhecida,Alta,Nenhuma,"15.000,00 a 35.000,00",Alto
2,C03,Desconhecida,Baixa,Nenhuma,"15.000,00 a 35.000,00",Moderado
3,C04,Desconhecida,Baixa,Nenhuma,"> 35.000,00",Alto
4,C05,Desconhecida,Baixa,Nenhuma,"> 35.000,00",Baixo
5,C06,Desconhecida,Baixa,Adequada,"> 35.000,00",Baixo
6,C07,Ruim,Baixa,Nenhuma,"< 15.000,00",Alto
7,C08,Ruim,Baixa,Adequada,"> 35.000,00",Moderado
8,C09,Boa,Baixa,Nenhuma,"> 35.000,00",Baixo
9,C10,Boa,Alta,Adequada,"> 35.000,00",Baixo


In [ ]:
df.iloc[:, 4]

,renda_anual
0,"< 15.000,00"
1,"15.000,00 a 35.000,00"
2,"15.000,00 a 35.000,00"
3,"> 35.000,00"
4,"> 35.000,00"
5,"> 35.000,00"
6,"< 15.000,00"
7,"> 35.000,00"
8,"> 35.000,00"
9,"> 35.000,00"


In [ ]:
lista = [10, 20, 30, 40, 50, 60, 70]
print(lista[2:5]) # slicing ou fatiamento de sequências
a, b, c = lista[2:5] # unpacking ou desempacotamento de sequências
print(a)
print(b)
print(c)
e, f, *g = ['a', 10, True, 10, 20] # unpacking ou desempacotamento de sequências
print(e)
print(f)
print(g)

[30, 40, 50]
30
40
50
a
10
[True, 10, 20]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   codigo_cliente    14 non-null     object
 1   historia_credito  14 non-null     object
 2   divida            14 non-null     object
 3   garantia          14 non-null     object
 4   renda_anual       14 non-null     object
 5   risco             14 non-null     object
dtypes: object(6)
memory usage: 800.0+ bytes


In [ ]:
df.describe().T

,count,unique,top,freq
codigo_cliente,14,14,C01,1
historia_credito,14,3,Desconhecida,5
divida,14,2,Alta,7
garantia,14,2,Nenhuma,11
renda_anual,14,3,"> 35.000,00",7
risco,14,3,Alto,6


In [ ]:
df['risco'].value_counts()

,count
risco,
Alto,6
Baixo,5
Moderado,3


In [ ]:
#Verifica se há valores Nulos/ausentes
df.isnull().sum()

,0
codigo_cliente,0
historia_credito,0
divida,0
garantia,0
renda_anual,0
risco,0


In [ ]:
#verifica se há valores duplicados
df.duplicated().sum()

0

###Limpeza dos dados

In [ ]:
df2 = df.iloc[:, 1:]

In [ ]:
df2

,historia_credito,divida,garantia,renda_anual,risco
0,Ruim,Alta,Nenhuma,"< 15.000,00",Alto
1,Desconhecida,Alta,Nenhuma,"15.000,00 a 35.000,00",Alto
2,Desconhecida,Baixa,Nenhuma,"15.000,00 a 35.000,00",Moderado
3,Desconhecida,Baixa,Nenhuma,"> 35.000,00",Alto
4,Desconhecida,Baixa,Nenhuma,"> 35.000,00",Baixo
5,Desconhecida,Baixa,Adequada,"> 35.000,00",Baixo
6,Ruim,Baixa,Nenhuma,"< 15.000,00",Alto
7,Ruim,Baixa,Adequada,"> 35.000,00",Moderado
8,Boa,Baixa,Nenhuma,"> 35.000,00",Baixo
9,Boa,Alta,Adequada,"> 35.000,00",Baixo


In [ ]:
df2.duplicated().sum()

0

###Pré-Processamento dos Dados


In [ ]:
x = df2.iloc[:, :4].values
x

array([['Ruim', 'Alta', 'Nenhuma', '< 15.000,00'],
       ['Desconhecida', 'Alta', 'Nenhuma', '15.000,00 a 35.000,00'],
       ['Desconhecida', 'Baixa', 'Nenhuma', '15.000,00 a 35.000,00'],
       ['Desconhecida', 'Baixa', 'Nenhuma', '> 35.000,00'],
       ['Desconhecida', 'Baixa', 'Nenhuma', '> 35.000,00'],
       ['Desconhecida', 'Baixa', 'Adequada', '> 35.000,00'],
       ['Ruim', 'Baixa', 'Nenhuma', '< 15.000,00'],
       ['Ruim', 'Baixa', 'Adequada', '> 35.000,00'],
       ['Boa', 'Baixa', 'Nenhuma', '> 35.000,00'],
       ['Boa', 'Alta', 'Adequada', '> 35.000,00'],
       ['Boa', 'Alta', 'Nenhuma', '< 15.000,00'],
       ['Boa', 'Alta', 'Nenhuma', '15.000,00 a 35.000,00'],
       ['Boa', 'Alta', 'Nenhuma', '> 35.000,00'],
       ['Ruim', 'Alta', 'Nenhuma', '15.000,00 a 35.000,00']], dtype=object)

In [ ]:
# Separando variáveis Explicativas (previsoras) e Variável TARGET (resultado)
# df[['historia_credito', 'divida', 'garantia', 'renda_anual']]
# x = df.iloc[:, 1:5].values

#df['risco']
y = df2.iloc[:, 4].values

print(f'Shape x: {x.shape} | y: {y.shape}')

Shape x: (14, 4) | y: (14,)


In [ ]:
y

array(['Alto', 'Alto', 'Moderado', 'Alto', 'Baixo', 'Baixo', 'Alto',
       'Moderado', 'Baixo', 'Baixo', 'Alto', 'Moderado', 'Baixo', 'Alto'],
      dtype=object)

In [ ]:
#atributos categóricos devem ser convertidos para atributos numéricos
from sklearn.preprocessing import LabelEncoder

label_encoder_historia = LabelEncoder()
label_encoder_divida = LabelEncoder()
label_encoder_garantia = LabelEncoder()
label_encoder_renda = LabelEncoder()

In [ ]:
x[:, 0]

array(['Ruim', 'Desconhecida', 'Desconhecida', 'Desconhecida',
       'Desconhecida', 'Desconhecida', 'Ruim', 'Ruim', 'Boa', 'Boa',
       'Boa', 'Boa', 'Boa', 'Ruim'], dtype=object)

In [ ]:
label_encoder_historia.fit_transform(x[:,0])

array([2, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 2])

In [ ]:
x[:,0] = label_encoder_historia.fit_transform(x[:,0])
x[:,1] = label_encoder_divida.fit_transform(x[:,1])
x[:,2] = label_encoder_garantia.fit_transform(x[:,2])
x[:,3] = label_encoder_renda.fit_transform(x[:,3])

In [ ]:
x

array([[2, 0, 1, 1],
       [1, 0, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 1, 2],
       [1, 1, 1, 2],
       [1, 1, 0, 2],
       [2, 1, 1, 1],
       [2, 1, 0, 2],
       [0, 1, 1, 2],
       [0, 0, 0, 2],
       [0, 0, 1, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 2],
       [2, 0, 1, 0]], dtype=object)

In [ ]:
#mostrar o nome das classes para 'historia_credito'
print(label_encoder_historia.classes_)

#mostrar o valor númerico para cada classe do atributo 'historia_credito'
print(label_encoder_historia.transform(label_encoder_historia.classes_))

['Boa' 'Desconhecida' 'Ruim']
[0 1 2]


In [ ]:
#mostrar o nome das classes para 'divida'
print(label_encoder_divida.classes_)

#mostrar o valor númerico para cada classe do atributo 'divida'
print(label_encoder_divida.transform(label_encoder_divida.classes_))

['Alta' 'Baixa']
[0 1]


In [ ]:
#mostrar o nome das classes para 'garantia'
print(label_encoder_garantia.classes_)

#mostrar o valor númerico para cada classe do atributo 'garantia'
print(label_encoder_garantia.transform(label_encoder_garantia.classes_))

['Adequada' 'Nenhuma']
[0 1]


In [ ]:
#mostrar o nome das classes para 'renda_anual'
print(label_encoder_renda.classes_)

#mostrar o valor númerico para cada classe do atributo 'renda_anual'
print(label_encoder_renda.transform(label_encoder_renda.classes_))

['15.000,00 a 35.000,00' '< 15.000,00' '> 35.000,00']
[0 1 2]


In [ ]:
x

array([[2, 0, 1, 1],
       [1, 0, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 1, 2],
       [1, 1, 1, 2],
       [1, 1, 0, 2],
       [2, 1, 1, 1],
       [2, 1, 0, 2],
       [0, 1, 1, 2],
       [0, 0, 0, 2],
       [0, 0, 1, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 2],
       [2, 0, 1, 0]], dtype=object)

In [ ]:
y

array(['Alto', 'Alto', 'Moderado', 'Alto', 'Baixo', 'Baixo', 'Alto',
       'Moderado', 'Baixo', 'Baixo', 'Alto', 'Moderado', 'Baixo', 'Alto'],
      dtype=object)

In [ ]:
# importando Pacote para Amostragem dos Dados
from sklearn.model_selection import train_test_split

In [ ]:
# Realizando Amostragem dos Dados
# base de teste = de 30% a 20%  / base de treino = de 70% a 80%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=7)

In [ ]:
x_train

array([[2, 0, 1, 1],
       [1, 0, 1, 0],
       [2, 0, 1, 0],
       [0, 1, 1, 2],
       [2, 1, 0, 2],
       [0, 0, 1, 1],
       [1, 1, 1, 2],
       [2, 1, 1, 1],
       [0, 0, 0, 2],
       [1, 1, 1, 2]], dtype=object)

In [ ]:
x_test

array([[0, 0, 1, 2],
       [0, 0, 1, 0],
       [1, 1, 1, 0],
       [1, 1, 0, 2]], dtype=object)

###Construindo a Máquina Preditiva

In [ ]:
#Importando Pacotes para Construção das Máquinas Preditivas com Machine Learning
from sklearn.naive_bayes import GaussianNB

*   fit significa 'encaixar', algoritmo será encaixado.
*   Treinamento é uma tabela de probabilidade
*   x: variáveis previsoras
*   y: é a variável que contém a resposta esperada



In [ ]:
#algoritmo é bastante simples - geração de uma tabela de probabilidade

#GaussianNB: nome de uma distribuição estatística usada para problemas genéricos - distribuição normal
maquina_gnb = GaussianNB()

#processo de treinamento da máquina preditiva, passando dados de treino
maquina_gnb.fit(x_train, y_train)



GaussianNB()

### Avaliando a Máquina Preditiva

In [ ]:
#Importando a Métrica de Avaliação
from sklearn.metrics import accuracy_score

###Informações sobre a variável resultado - 'risco'

In [ ]:
# Fazer previsões no conjunto de teste
y_pred_test = maquina_gnb.predict(x_test)
y_pred_test

array(['Baixo', 'Alto', 'Alto', 'Baixo'], dtype='<U8')

In [ ]:
y_test

array(['Baixo', 'Moderado', 'Moderado', 'Baixo'], dtype=object)

In [ ]:
# Calcular a acurácia
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f'Acurácia: {accuracy_test:.2f}')
#se acertar 100% os dados podem estar viciados


Acurácia: 0.50


In [ ]:
# Fazer previsões no conjunto de treino
y_pred_train = maquina_gnb.predict(x_train)
y_pred_train

array(['Alto', 'Alto', 'Alto', 'Baixo', 'Moderado', 'Alto', 'Baixo',
       'Alto', 'Baixo', 'Baixo'], dtype='<U8')

In [ ]:
y_train

array(['Alto', 'Alto', 'Alto', 'Baixo', 'Moderado', 'Alto', 'Alto',
       'Alto', 'Baixo', 'Baixo'], dtype=object)

In [ ]:
# Calcular a acurácia
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f'Acurácia: {accuracy_train:.2f}')
#se acertar 100% os dados podem estar viciados

Acurácia: 0.90


In [ ]:
#não é possível visualizar a tabela de probabilidade

print(maquina_gnb.classes_)
print(maquina_gnb.class_count_)
print(maquina_gnb.class_prior_) #porcentagem da base de dados


['Alto' 'Baixo' 'Moderado']
[6. 3. 1.]
[0.6 0.3 0.1]


###Prevendo/classificando um novo cliente

In [ ]:
'''
Cliente 1:
historia_credito = 0 (Boa)
divida = 0 (Alta)
garantia = 1 (Nenhuma)
renda = 2 (> 35.000,00)

Cliente 2:
historia_credito = 2 (Ruim)
divida = 0 (Alta)
garantia = 0 (Adequada)
renda = 1 (< 15.000,00)
'''

previsao = maquina_gnb.predict([[0, 0, 1, 2],
                                [2, 0, 0, 1]])

#predict retorna a classe não a porcentagem
previsao

array(['Baixo', 'Alto'], dtype='<U8')

In [ ]:
label_encoder_historia.classes_

array(['Boa', 'Desconhecida', 'Ruim'], dtype=object)

In [ ]:
hc = label_encoder_historia.transform(['Ruim'])
hc

array([2])

## Como salvar e depois carregar nossa Máquina Preditiva

In [ ]:
#Comando de Salvamento da Máquina Preditiva
import pickle

# .pkl é a extensão  mode = wb modo escrita
with open('maquina_preditiva.pkl', mode = 'wb') as pickle_out:
    pickle.dump(maquina_gnb, pickle_out)


In [ ]:
# Carregar o modelo treinado
with open('maquina_preditiva.pkl', 'rb') as pickle_in:
    maquina_carregada = pickle.load(pickle_in)

# Agora, você pode usar maquina_carregada para fazer previsões


In [ ]:
maquina_carregada.predict([[2, 1, 0, 0]])

array(['Alto'], dtype='<U8')

##Outros Algoritmos Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

maquina_mnb = MultinomialNB(force_alpha=True)
maquina_bnb = BernoulliNB(force_alpha=True)


maquina_mnb.fit(x_train, y_train)
maquina_bnb.fit(x_train, y_train)

BernoulliNB()

O parâmetro alpha no contexto de classificadores Naive Bayes é um hiperparâmetro de suavização (smoothing). Ele é usado para lidar com a situação em que certas palavras ou recursos não aparecem em todas as classes durante o treinamento. A suavização ajuda a evitar probabilidades condicionais de zero, o que pode causar problemas durante a classificação.

In [ ]:
y_gnb = maquina_gnb.predict(x_test)
y_mnb = maquina_mnb.predict(x_test)
y_bnb = maquina_bnb.predict(x_test)

In [ ]:
print("Gaussian = ",accuracy_score(y_test,y_gnb))
print("Multinomial = ",accuracy_score(y_test,y_mnb))
print("Bernoulli = ",accuracy_score(y_test,y_bnb))

Gaussian =  0.5
Multinomial =  0.5
Bernoulli =  0.25
